<a href="https://colab.research.google.com/github/Nithya9404/Naan_mudalvan_GEN_AI/blob/main/Machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/datasets/deu.txt',sep='\t')
df.head(5)

,Go.,Geh.,CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8597805 (Roujin)
0,Hi.,Hallo!,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
1,Hi.,Grüß Gott!,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
2,Run!,Lauf!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
3,Run.,Lauf!,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
4,Wow!,Potzdonner!,CC-BY 2.0 (France) Attribution: tatoeba.org #5...


In [ ]:
data_path = '/content/drive/MyDrive/datasets/deu.txt'

In [ ]:
df.columns =["english","german","text"]
df.head(10)

,english,german,text
0,Hi.,Hallo!,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
1,Hi.,Grüß Gott!,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
2,Run!,Lauf!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
3,Run.,Lauf!,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
4,Wow!,Potzdonner!,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
5,Wow!,Donnerwetter!,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
6,Fire!,Feuer!,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
7,Help!,Hilfe!,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
8,Help!,Zu Hülf!,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
9,Stop!,Stopp!,CC-BY 2.0 (France) Attribution: tatoeba.org #4...


In [ ]:
df.drop('text',axis=1,inplace=True)

In [ ]:
df.head(10)

,english,german
0,Hi.,Hallo!
1,Hi.,Grüß Gott!
2,Run!,Lauf!
3,Run.,Lauf!
4,Wow!,Potzdonner!
5,Wow!,Donnerwetter!
6,Fire!,Feuer!
7,Help!,Hilfe!
8,Help!,Zu Hülf!
9,Stop!,Stopp!


In [ ]:
!pip install tensorflow

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense

batch_size=64
epochs=100
latent_dim=256
num_samples=10000


In [ ]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [ ]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))

num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)

max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])

In [ ]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 85
Max sequence length for inputs: 15
Max sequence length for outputs: 45


In [ ]:
input_token_index=dict(
    [(char,i) for i, char in enumerate(input_characters)])
target_token_index=dict(
[(char,i) for i, char in enumerate(target_characters)])

In [ ]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [ ]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 49s 366ms/step - loss: 1.5685 - accuracy: 0.6361 - val_loss: 1.4385 - val_accuracy: 0.6003
Epoch 2/100
125/125 [==============================] - 43s 345ms/step - loss: 1.2188 - accuracy: 0.6637 - val_loss: 1.2573 - val_accuracy: 0.6477
Epoch 3/100
125/125 [==============================] - 41s 330ms/step - loss: 1.0409 - accuracy: 0.7135 - val_loss: 1.0609 - val_accuracy: 0.7200
Epoch 4/100
125/125 [==============================] - 41s 327ms/step - loss: 0.8797 - accuracy: 0.7534 - val_loss: 0.9284 - val_accuracy: 0.7471
Epoch 5/100
125/125 [==============================] - 40s 324ms/step - loss: 0.8030 - accuracy: 0.7731 - val_loss: 0.8713 - val_accuracy: 0.7599
Epoch 6/100
125/125 [==============================] - 39s 314ms/step - loss: 0.7827 - accuracy: 0.7808 - val_loss: 0.8310 - val_accuracy: 0.7672
Epoch 7/100
125/125 [==============================] - 42s 332ms/step - loss: 0.7221 - accuracy: 0.7917 - val_loss: 0.8152 -

In [ ]:
model.save('eng-german.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = [h, c]
    return decoded_sentence

In [ ]:
for i in range(5):
  input_seq = encoder_input_data[i:i+1]
  decoded_sentence = decode_sequence(input_seq)
  print('-')
  print("Input: ",input_texts[i])
  print("Decoded sentence: ",decoded_sentence)

1/1 [==============================] - 0s 25ms/step
-
Input:  Go.
Decoded sentence:  Mach wieder stehlen.

1/1 [==============================] - 0s 36ms/step
-
Input:  Hi.
Decoded sentence:  Hallo!

1/1 [==============================] - 0s 37ms/step
-
Input:  Hi.
Decoded sentence:  Hallo!

1/1 [==============================] - 0s 37ms/step
-
Input:  Run!
Decoded sentence:  Vergiss mir!

1/1 [==============================] - 0s 23ms/step
-
Input:  Run.
Decoded sentence:  Mach wes!

